# 1. Import Ntuple and DecayHash

In [1]:
import root_pandas
import uproot
import decayHash
import basf2 as b2
from decayHash import DecayHashMap
import sys
import numpy as np
import ROOT
import pandas

# read in root-file as a pandas dataframe
filename='mixed_bengal_e_2.root'
decayhash='hashmap_mixed_bengal_e_2.root'
#data = uproot.open(filename)['B0'].arrays(library="pd")
data = root_pandas.read_root(filename,key='B0')
hashmap = DecayHashMap(decayhash, removeRadiativeGammaFlag=False)
hashmap2 = DecayHashMap(decayhash, removeRadiativeGammaFlag=True)

Welcome to JupyROOT 6.24/00


In [2]:
data['B0_mcPDG'] = data['B0_mcPDG'].fillna(0)
data['isSignal'] = 1.0
data.eval('B_D_ReChi2 = B0_vtxReChi2 + D_vtxReChi2', inplace=True)
data.eval('p_D_l = D_CMS_p + e_CMS_p', inplace=True)

# Calculate the q2 in different ways
#data.eval('q2_SigOnly = (0.5*Ecms - D_CMS_E)**2 - D_CMS_px**2 - D_CMS_py**2 - D_CMS_pz**2', inplace=True)
#data.eval('MM2_SigOnly = (0.5*Ecms - D_CMS_E - e_CMS_E)**2 - (D_CMS_px + e_CMS_px)**2 - (D_CMS_py + e_CMS_py)**2 - (D_CMS_pz + e_CMS_pz)**2', inplace=True)

In [3]:
data.nElectrons90.value_counts()

1    2225987
2     626575
3      65071
4       3382
5         96
6          3
Name: nElectrons90, dtype: int64

In [4]:
data.B0_roeMC_MissFlags_my_mask.value_counts()

9.0       1580
1.0       1299
521.0      669
513.0      561
1025.0     536
          ... 
1674.0       1
355.0        1
795.0        1
129.0        1
1567.0       1
Name: B0_roeMC_MissFlags_my_mask, Length: 281, dtype: int64

In [ ]:
import pandas
pandas.set_option('display.max_rows', None)
print(data.isna().sum())

# 2. Add DecayMode column to the Ntuple

In [3]:
from collections import OrderedDict

def found(modes,row):
    for mode in modes:
        # check the decay chain for the reconstructed B meson only
        if mode.startswith(str(int(row['B0_mcPDG']))):
            decaytree = ROOT.Belle2.DecayTree(mode)
            if hashmap2.get_original_decay(row["B0_DecayHash"],row["B0_DecayHashEx"]).find_decay(decaytree):
                return True
        else:
            continue
    return False

def decay_mode(row):
    for name,modes in mode_dict.items():
        if found(modes,row):
            return name
    return 'bkg'

# the order of keys might be important, try to keep the muon modes at the bottom for e reconstruction
# the e modes will be kept at the bottom for a muon reconstruction
mode_dict = OrderedDict()
mode_dict['sig_D_tau_nu']=['511 (-> -411 (-> 321 -211 -211) -15 (-> -11 12 -16) 16)',
                           '-511 (-> 411 (-> -321 211 211) 15 (-> 11 -12 16) -16)',
                           '511 (-> -411 (-> 321 -211 -211) -15 (-> -13 14 -16) 16)',
                           '-511 (-> 411 (-> -321 211 211) 15 (-> 13 -14 16) -16)']

mode_dict['sig_D_e_nu']=['511 (-> -411 (-> 321 -211 -211) -11 12)',
                         '-511 (-> 411 (-> -321 211 211) 11 -12)']

mode_dict['sig_Dst_tau_nu']=['511 (-> -413 (-> -411 (-> 321 -211 -211) 111) -15 (-> -11 12 -16) 16)',
                             '-511 (-> 413 (-> 411 (-> -321 211 211) 111) 15 (-> 11 -12 16) -16)',
                             '511 (-> -413 (-> -411 (-> 321 -211 -211) 111) -15 (-> -13 14 -16) 16)',
                             '-511 (-> 413 (-> 411 (-> -321 211 211) 111) 15 (-> 13 -14 16) -16)',
                             '511 (-> -413 (-> -411 (-> 321 -211 -211) 22) -15 (-> -11 12 -16) 16)',
                             '-511 (-> 413 (-> 411 (-> -321 211 211) 22) 15 (-> 11 -12 16) -16)',
                             '511 (-> -413 (-> -411 (-> 321 -211 -211) 22) -15 (-> -13 14 -16) 16)',
                             '-511 (-> 413 (-> 411 (-> -321 211 211) 22) 15 (-> 13 -14 16) -16)']

mode_dict['sig_Dst_e_nu']=['511 (-> -413 (-> -411 (-> 321 -211 -211) 111) -11 12)',
                           '511 (-> -413 (-> -411 (-> 321 -211 -211) 22) -11 12)',
                           '-511 (-> 413 (-> 411 (-> -321 211 211) 111) 11 -12)',
                           '-511 (-> 413 (-> 411 (-> -321 211 211) 22) 11 -12)']

mode_dict['all_Dstst_tau_nu']=['511 (-> -10413 -15 (-> -11 12 -16) 16)','-511 (-> 10413 15 (-> 11 -12 16) -16)',
                               '511 (-> -10411 -15 (-> -11 12 -16) 16)','-511 (-> 10411 15 (-> 11 -12 16) -16)',
                               '511 (-> -20413 -15 (-> -11 12 -16) 16)','-511 (-> 20413 15 (-> 11 -12 16) -16)',
                               '511 (-> -415 -15 (-> -11 12 -16) 16)',  '-511 (-> 415 15 (-> 11 -12 16) -16)',
                               '521 (-> -10423 -15 (-> -11 12 -16) 16)','-521 (-> 10423 15 (-> 11 -12 16) -16)',
                               '521 (-> -10421 -15 (-> -11 12 -16) 16)','-521 (-> 10421 15 (-> 11 -12 16) -16)',
                               '521 (-> -20423 -15 (-> -11 12 -16) 16)','-521 (-> 20423 15 (-> 11 -12 16) -16)',
                               '521 (-> -425 -15 (-> -11 12 -16) 16)',  '-521 (-> 425 15 (-> 11 -12 16) -16)',
                               '511 (-> -10413 -15 (-> -13 14 -16) 16)','-511 (-> 10413 15 (-> 13 -14 16) -16)',
                               '511 (-> -10411 -15 (-> -13 14 -16) 16)','-511 (-> 10411 15 (-> 13 -14 16) -16)',
                               '511 (-> -20413 -15 (-> -13 14 -16) 16)','-511 (-> 20413 15 (-> 13 -14 16) -16)',
                               '511 (-> -415 -15 (-> -13 14 -16) 16)',  '-511 (-> 415 15 (-> 13 -14 16) -16)',
                               '521 (-> -10423 -15 (-> -13 14 -16) 16)','-521 (-> 10423 15 (-> 13 -14 16) -16)',
                               '521 (-> -10421 -15 (-> -13 14 -16) 16)','-521 (-> 10421 15 (-> 13 -14 16) -16)',
                               '521 (-> -20423 -15 (-> -13 14 -16) 16)','-521 (-> 20423 15 (-> 13 -14 16) -16)',
                               '521 (-> -425 -15 (-> -13 14 -16) 16)',  '-521 (-> 425 15 (-> 13 -14 16) -16)']

mode_dict['all_Dstst_e_nu']=['511 (-> -10413 -11 12)','-511 (-> 10413 11 -12)',
                             '511 (-> -10411 -11 12)','-511 (-> 10411 11 -12)',
                             '511 (-> -20413 -11 12)','-511 (-> 20413 11 -12)',
                             '511 (-> -415 -11 12)',  '-511 (-> 415 11 -12)',
                             '511 (-> -411 221 -11 12)','-511 (-> 411 221 11 -12)',
                             '511 (-> -411 111 -11 12)','-511 (-> 411 111 11 -12)',
                             '511 (-> -411 111 111 -11 12)','-511 (-> 411 111 111 11 -12)',
                             '511 (-> -411 211 -211 -11 12)','-511 (-> 411 211 -211 11 -12)',
                             '511 (-> -413 221 -11 12)','-511 (-> 413 221 11 -12)',
                             '511 (-> -413 111 -11 12)','-511 (-> 413 111 11 -12)',
                             '511 (-> -413 111 111 -11 12)','-511 (-> 413 111 111 11 -12)',
                             '511 (-> -413 211 -211 -11 12)','-511 (-> 413 211 -211 11 -12)',
                             '511 (-> -421 -211 -11 12)','-511 (-> 421 211 11 -12)',
                             '511 (-> -423 -211 -11 12)','-511 (-> 423 211 11 -12)',
                             '521 (-> -10423 -11 12)','-521 (-> 10423 11 -12)',
                             '521 (-> -10421 -11 12)','-521 (-> 10421 11 -12)',
                             '521 (-> -20423 -11 12)','-521 (-> 20423 11 -12)',
                             '521 (-> -425 -11 12)',  '-521 (-> 425 11 -12)',
                             '521 (-> -411 211 -11 12)','-521 (-> 411 211 11 -12)',
                             '521 (-> -411 211 111 -11 12)','-521 (-> 411 211 111 11 -12)',
                             '521 (-> -413 211 -11 12)','-521 (-> 413 211 11 -12)',
                             '521 (-> -413 211 111 -11 12)','-521 (-> 413 211 111 11 -12)']

mode_dict['sig_D_mu_nu']=['511 (-> -411 (-> 321 -211 -211) -13 14)',
                          '-511 (-> 411 (-> -321 211 211) 13 -14)']

mode_dict['sig_Dst_mu_nu']=['511 (-> -413 (-> -411 (-> 321 -211 -211) 111) -13 14)',
                           '511 (-> -413 (-> -411 (-> 321 -211 -211) 22) -13 14)',
                           '-511 (-> 413 (-> 411 (-> -321 211 211) 111) 13 -14)',
                           '-511 (-> 413 (-> 411 (-> -321 211 211) 22) 13 -14)']

mode_dict['all_Dstst_mu_nu']=['511 (-> -10413 -13 14)','-511 (-> 10413 13 -14)',
                              '511 (-> -10411 -13 14)','-511 (-> 10411 13 -14)',
                              '511 (-> -20413 -13 14)','-511 (-> 20413 13 -14)',
                              '511 (-> -415 -13 14)',  '-511 (-> 415 13 -14)',
                              '511 (-> -411 221 -13 14)','-511 (-> 411 221 13 -14)',
                              '511 (-> -411 111 -13 14)','-511 (-> 411 111 13 -14)',
                              '511 (-> -411 111 111 -13 14)','-511 (-> 411 111 111 13 -14)',
                              '511 (-> -411 211 -211 -13 14)','-511 (-> 411 211 -211 13 -14)',
                              '511 (-> -413 221 -13 14)','-511 (-> 413 221 13 -14)',
                              '511 (-> -413 111 -13 14)','-511 (-> 413 111 13 -14)',
                              '511 (-> -413 111 111 -13 14)','-511 (-> 413 111 111 13 -14)',
                              '511 (-> -413 211 -211 -13 14)','-511 (-> 413 211 -211 13 -14)',
                              '511 (-> -421 -211 -13 14)','-511 (-> 421 211 13 -14)',
                              '511 (-> -423 -211 -13 14)','-511 (-> 423 211 13 -14)',
                              '521 (-> -10423 -13 14)','-521 (-> 10423 13 -14)',
                              '521 (-> -10421 -13 14)','-521 (-> 10421 13 -14)',
                              '521 (-> -20423 -13 14)','-521 (-> 20423 13 -14)',
                              '521 (-> -425 -13 14)',  '-521 (-> 425 13 -14)',
                              '521 (-> -411 211 -13 14)','-521 (-> 411 211 13 -14)',
                              '521 (-> -411 211 111 -13 14)','-521 (-> 411 211 111 13 -14',
                              '521 (-> -413 211 -13 14)','-521 (-> 413 211 13 -14)',
                              '521 (-> -413 211 111 -13 14)','-521 (-> 413 211 111 13 -14)']

In [4]:
data['DecayMode'] = data.apply(decay_mode, axis=1).astype('category') #axis=0 will allow the application to be done at a column

data.DecayMode.value_counts()

bkg                 2570799
sig_D_e_nu           343144
sig_Dst_e_nu         259099
all_Dstst_e_nu       151355
sig_D_tau_nu          13512
sig_Dst_tau_nu         8720
all_Dstst_tau_nu       4393
all_Dstst_mu_nu        1114
sig_D_mu_nu              83
sig_Dst_mu_nu            47
Name: DecayMode, dtype: int64

In [5]:
for idx, chunk in enumerate(np.array_split(data, 7)):
    chunk.to_root(f'subset_{idx}.root', key='B0')

In [ ]:
cut='DecayMode=="sig_D_e_nu" and B0_isContinuumEvent!=1'
print()
candidate12 = data.query(cut).iloc[2][['B0_DecayHash', "B0_DecayHashEx"]].values

# print the original decay as simulated in MC with removed Bremsstrahlung gammas
print("Monte Carlo Decay with removed Bremsstrahlung gammas: ")
org2 = hashmap2.get_original_decay(*candidate12)
print(org2.to_string())

In [6]:
samples = {}
names = ['BC','AC']
cut = 'D_vtxReChi2<13 and B0_vtxReChi2<14 and 5<B0_roeMbc_my_mask and \
    -5<B0_roeDeltae_my_mask<2 and 4.3<B0_CMS2_weMbc and -3<B0_CMS0_weDeltae<2 and \
    abs(B0_roeCharge_my_mask)<3 and -3.2<B0_deltaE<0 and e_CMS_p>0.2 and \
    nElectrons90+nMuons90==1'
for name in names:
    if name == 'BC':
        df = data.copy()
    else:
        df = data.query(cut).copy()
    
    print(f'{name} before BCS')
    print(df.DecayMode.value_counts())

    df_bestSelected=df.loc[df.groupby(['__experiment__','__run__','__event__','__production__']).B_D_ReChi2.idxmin()]

    print(f'{name} after BCS')
    print(df_bestSelected.DecayMode.value_counts())
    
    samples[name] = df_bestSelected

BC before BCS
bkg                 2525324
sig_D_tau_nu         145484
sig_Dst_tau_nu        91813
sig_D_e_nu            66257
sig_Dst_e_nu          50851
all_Dstst_e_nu        39694
all_Dstst_tau_nu       1222
all_Dstst_mu_nu         441
sig_D_mu_nu              14
sig_Dst_mu_nu            14
Name: DecayMode, dtype: int64
BC after BCS
bkg                 1933953
sig_D_tau_nu         138148
sig_Dst_tau_nu        87010
sig_D_e_nu            64037
sig_Dst_e_nu          49002
all_Dstst_e_nu        36481
all_Dstst_tau_nu       1120
all_Dstst_mu_nu         375
sig_D_mu_nu              13
sig_Dst_mu_nu            12
Name: DecayMode, dtype: int64
AC before BCS
bkg                 536693
sig_D_tau_nu         89075
sig_Dst_tau_nu       55736
sig_D_e_nu           40338
sig_Dst_e_nu         30776
all_Dstst_e_nu       17511
all_Dstst_tau_nu       550
all_Dstst_mu_nu         66
sig_D_mu_nu              4
sig_Dst_mu_nu            3
Name: DecayMode, dtype: int64
AC after BCS
bkg                 475081